# Analyze based on semantic categories

1.) change tfidf so we compare equivalent categories only - done
2.) update ranking accordingly

In [13]:
import os
from collections import Counter, defaultdict
import csv
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_fscore_support

In [3]:
f_original = os.listdir('../contexts/giga_full/vocab')
print(len(f_original))
f_update = os.listdir('../contexts/giga_full_updated/vocab')
print(len(f_update))

1446
1636


In [2]:
f_original = os.listdir('../contexts/wiki/vocab')
print(len(f_original))
f_update = os.listdir('../contexts/wiki_updated/vocab')
print(len(f_update))

1669
1874


In [19]:

def get_categories(prop, model_name):
    analysis_type = 'tfidf-raw-10000/each_target_vs_corpus_per_category'
    path_dir = f'../results/{model_name}/{analysis_type}'
    path_dir = f'{path_dir}/{prop}'
    categories = set()
    for d in os.listdir(path_dir):
        categories.add(d)
    return categories

def get_context_cnts(prop, cat, label, model_name):
    
    analysis_type = 'tfidf-raw-10000/each_target_vs_corpus_per_category'
    path_dir = f'../results/{model_name}/{analysis_type}'
    path_dir = f'{path_dir}/{prop}'
    path_label = f'{path_dir}/{cat}/{label}'
    
    context_cnt = Counter()
    for f in os.listdir(path_label):
        full_path = f'{path_label}/{f}'
        if full_path.endswith('.csv'):
            with open(full_path) as infile:
                data = list(csv.DictReader(infile))
            for d in data:
                context = d['']
                diff = float(d['diff'])
                if diff > 0:
                    context_cnt[context] += 1
    return context_cnt
    
def get_n_concepts_total(prop, cat, model_name):
    
    analysis_type = 'tfidf-raw-10000/each_target_vs_corpus_per_category'
    path_dir = f'../results/{model_name}/{analysis_type}'
    path_dir = f'{path_dir}/{prop}'
    label = 'pos'
    path_pos = f'{path_dir}/{cat}/{label}'
    label = 'neg'
    path_neg = f'{path_dir}/{cat}/{label}'
    
    files_pos = [f for f in os.listdir(path_pos) if f.endswith('.csv')]
    files_neg = [f for f in os.listdir(path_neg) if f.endswith('.csv')]
    
    return len(files_pos), len(files_neg)

def get_f1_distinctiveness(n_pos, n_neg, total_pos, total_neg):
    
   
    total_instances = total_pos + total_neg
    labels = []
    [labels.append('pos') for i in range(total_pos)]
    [labels.append('neg') for i in range(total_neg)]
    pred_labels_pos = []
    for i in range(total_pos):
        if i < n_pos:
            pred_labels_pos.append('pos')
        else:
            pred_labels_pos.append('neg')
#     print(n_pos, total_pos)
#     print(pred_labels_pos.count('pos'), pred_labels_pos.count('neg'))
    
    pred_labels_neg = []
    for i in range(total_neg):
        if i < n_neg:
            pred_labels_neg.append('pos')
        else:
            pred_labels_neg.append('neg')
#     print(n_neg, total_neg)
#     print(pred_labels_neg.count('pos'), pred_labels_neg.count('neg'))
    
    predictions = pred_labels_pos + pred_labels_neg
    
    
    #print(len(labels), len(predictions))
    #print(pos_predictions, neg_predictions)
    
    p, r, f1, supp = precision_recall_fscore_support(labels, predictions, average = 'weighted', 
                                                     zero_division=0)
    #average='weighted'
    
    return p, r, f1


    
def aggregate_contexts(prop, cutoff, model_name):
    aggregation_name = 'aggregated-tfidf-raw-10000-categories'
    path_dir_agg = f'../analysis/{model_name}/{aggregation_name}/{prop}'
    os.makedirs(path_dir_agg, exist_ok = True)
    
    context_cnts_all = Counter()
    context_cat_dict = defaultdict(set)

    cats = get_categories(prop, model_name)

    for cat in cats:
        context_cnts_pos = get_context_cnts(prop, cat, 'pos', model_name)
        context_cnts_neg = get_context_cnts(prop, cat, 'neg', model_name)
        total_pos, total_neg = get_n_concepts_total(prop, cat, model_name)
        
        context_f1_dict = Counter()
        context_score_dict = defaultdict(dict)
        
        # get distinctiveness
        for c, cnt_pos in context_cnts_pos.most_common():
            cnt_neg = context_cnts_neg[c]
            p, r, f1 = get_f1_distinctiveness(cnt_pos, cnt_neg, total_pos, total_neg)
            context_f1_dict[c] = f1
            context_score_dict[c] = {'p': p,'r':r, 'f1': f1}
        
        table = []
        for c, f1 in context_f1_dict.most_common():
            scores = context_score_dict[c]
            d = dict()
            d['context'] = c
            d.update(scores)
            d['n_pos'] = context_cnts_pos[c]
            d['total_pos'] = total_pos
            d['n_neg'] = context_cnts_neg[c]
            d['total_neg'] = total_neg
            table.append(d)
        
        # collect and write to file
        f = f'{path_dir_agg}/{cat}.csv'
        
        header = table[0].keys()
        with open(f, 'w') as outfile:
            writer = csv.DictWriter(outfile, fieldnames = header)
            writer.writeheader()
            for d in table:
                writer.writerow(d)
        
                
def prepare_annotation(prop, model_name, cutoff=3, cutoff_concepts = 5):
    
    annotation_name = f'annotation-tfidf-top_{cutoff}_{cutoff_concepts}-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    os.makedirs(path_dir_annotation, exist_ok = True)
    f_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}/annotation-updated.csv'
    
    # paths aggregated files:
    aggregation_name = 'aggregated-tfidf-raw-10000-categories'
    path_dir_agg = f'../analysis/{model_name}/{aggregation_name}/{prop}'

    
    # get categories
    cats = get_categories(prop, model_name)
    
    # collect all contexts and categories 
    context_cats_dict = defaultdict(set)
    
    # load top per category
    for cat in cats:
        path = f'{path_dir_agg}/{cat}.csv'
        with open(path) as infile:
            data = list(csv.DictReader(infile))
        # sort by f1
        f1_dict  = defaultdict(list)
        for d in data:
            f1 = d['f1']
            f1_dict[f1].append(d)
        scores = sorted(list(f1_dict.keys()), reverse=True)
        top_scores = scores[:cutoff]
        top_context_dicts = []
        for ts in top_scores:
            dicts = f1_dict[ts]
            for d in dicts:
                n_pos = int(d['n_pos'])
                if n_pos > cutoff_concepts:
                    top_context_dicts.append(d)
    
        contexts = [d['context'] for d in top_context_dicts]
        # record categories
        for c in contexts:
            context_cats_dict[c].add(cat)
    
    with open(f_annotation, 'w') as outfile:
        outfile.write('context,evidence_type,categories\n')
        for c, cats in context_cats_dict.items():
            outfile.write(f'{c}, ,{" ".join(cats)}\n')

def get_properties():
    properties = []
    for path in os.listdir('../data/aggregated/'):
        prop = path.split('.')[0]
        if 'female-' not in prop and prop != '':
            properties.append(prop)
    return properties

def get_top_distinctive_contexts(properties, model_name):
    aggregation_name = 'aggregated-tfidf-raw-10000-categories'
    table = []
    for prop in properties:
        path_dir_agg = f'../analysis/{model_name}/{aggregation_name}/{prop}'
        path = path = f'{path_dir_agg}/all.csv'
        # load file containing all concepts and simply load first one
        with open(path) as infile:
            data = list(csv.DictReader(infile))
        # top distinctive context
        d_prop = dict()
        d_prop['property'] = prop
        # sort data by f1
        f1_dict = defaultdict(list)
        for d in data:
            f1 = d['f1']
            f1_dict[f1].append(d)
        
        top_score = max(list(f1_dict.keys()))
        top_dicts = f1_dict[top_score]
        top_context_dict = top_dicts[0]
        top_contexts = ' '.join([d['context'] for d in top_dicts])
        

        for k, v in top_context_dict.items():
            if k != 'context':
                v = float(v)
                d_prop[k] = v
        d_prop['contexts'] = top_contexts
        table.append(d_prop)
    return table

In [22]:
model_name = 'giga_full_updated'
properties = get_properties()
properties_test = ['dangerous', 'cold', 'lay_eggs']
properties = [p for p in properties if p not in properties_test]
#properties = properties_test
cutoff = 3
cutoff_concepts = 3

for prop in properties:
    print(prop)
    aggregate_contexts(prop, cutoff, model_name)
    prepare_annotation(prop, model_name, cutoff, cutoff_concepts) 

square
warm
black
red
fly
wings
sweet
hot
used_in_cooking
juicy
green
made_of_wood
blue
yellow
roll
female
round
wheels
swim


In [4]:
# get top distinctive contexts per prop

model_name = 'giga_full_updated'
properties = get_properties()
table = get_top_distinctive_contexts(properties, model_name)
df = pd.DataFrame(table)
df.sort_values('f1', ascending = False).round(2)

,property,p,r,f1,n_pos,total_pos,n_neg,total_neg,contexts
9,used_in_cooking,0.96,0.95,0.95,93.0,101.0,0.0,56.0,add recipe
4,fly,0.88,0.87,0.87,29.0,44.0,2.0,90.0,flew
0,square,0.91,0.84,0.86,70.0,87.0,0.0,21.0,built
20,lay_eggs,0.86,0.84,0.83,20.0,33.0,1.0,57.0,eggs
16,female,0.85,0.84,0.83,77.0,109.0,9.0,144.0,herself
7,sweet,0.87,0.82,0.83,65.0,91.0,1.0,63.0,sweet
5,dangerous,0.86,0.82,0.82,45.0,65.0,1.0,51.0,killed
13,blue,0.87,0.83,0.81,31.0,59.0,0.0,106.0,magic
6,wings,0.85,0.82,0.81,36.0,60.0,1.0,77.0,bird
19,wheels,0.87,0.79,0.80,51.0,70.0,1.0,25.0,drove wheel


### Transfer old annotations to new files


In [4]:
properties = get_properties()
model_name_current = 'giga_full_updated'
model_name_old = 'giga_full'


for prop in properties:
    # current file:
    annotation_name = 'annotation-tfidf-top20-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    f_annotation_new = f'{path_dir_annotation}/annotation.csv'
    f_annotation_tr = f'{path_dir_annotation}/annotation-transferred.csv'

    # old file:
    annotation_name = 'annotation-tfidf-top20-raw-10000'
    path_dir_annotation = f'../analysis/{model_name_old}/{annotation_name}/{prop}-pos'
    f_annotation_old = f'{path_dir_annotation}/annotation-done.csv'

    # load old annotations
    context_annotation_dict=dict()
    with open(f_annotation_old) as infile:
        data = list(csv.DictReader(infile))
        for d in data:
            c = d['context']
            et = d['evidence']
            context_annotation_dict[c] = et
            #c = d['context']

    # load new candidates

    with open(f_annotation_new) as infile:
        data = list(csv.DictReader(infile))

    # fill in old annotations
    for d in data:
        c = d['context']
        if c in context_annotation_dict:
            et = context_annotation_dict[c]
        else:
            et = 'NA'
        d['evidence_type'] = et

    # write to new file

    with open(f_annotation_tr, 'w') as outfile:
        writer = csv.DictWriter(outfile, fieldnames = data[0].keys())
        writer.writeheader()
        for d in data:
            writer.writerow(d)

In [29]:
# transfer new annotations to updated f1 scores

properties = get_properties()
#properties = ['dangerous']
model_name_current = 'giga_full_updated'
model_name_old = 'giga_full'


for prop in properties:
    # current file:
    annotation_name = 'annotation-tfidf-top_3_3-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    f_annotation_new = f'{path_dir_annotation}/annotation-updated.csv'
    f_annotation_tr = f'{path_dir_annotation}/annotation-transferred-updated.csv'

    # old file:
    annotation_name = 'annotation-tfidf-top_3_5-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    f_annotation_old = f'{path_dir_annotation}/annotation-updated-done.csv'

    # load old annotations
    if os.path.isfile(f_annotation_old):
        print('found file')
        context_annotation_dict=dict()
        with open(f_annotation_old) as infile:
            data = list(csv.DictReader(infile))
            for d in data:
                c = d['context']
                et = d['evidence_type']
                context_annotation_dict[c] = et
                #c = d['context']

        # load new candidates

        with open(f_annotation_new) as infile:
            data = list(csv.DictReader(infile))

        # fill in old annotations
        for d in data:
            c = d['context']
            if c in context_annotation_dict:
                et = context_annotation_dict[c]
            else:
                et = 'NA'
            d['evidence_type'] = et

        # write to new file

        with open(f_annotation_tr, 'w') as outfile:
            writer = csv.DictWriter(outfile, fieldnames = data[0].keys())
            writer.writeheader()
            for d in data:
                writer.writerow(d)

found file
found file
found file


### Complete annotations

In [4]:
from collections import defaultdict
import os
import csv

In [36]:
def get_annotation_status(model_name, top_cutoff, concept_cutoff):
    dir_path = f'../analysis/{model_name}'
    dir_annotations = f'{dir_path}/annotation-tfidf-top_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    annotation_dict = defaultdict(set)

    for f in os.listdir(dir_annotations):
        if  not f.endswith('.csv') and not f.endswith('.ipynb_checkpoints'):
            prop = f.split('/')[-1]
            full_path = f'{dir_annotations}/{f}'
            #print(full_path)
            # get categories:
            files = os.listdir(full_path)
            if 'annotation-updated-done.csv' in files:
                annotation_dict['complete'].add(prop)
            else:
                annotation_dict['incomplete'].add(prop)
    return annotation_dict

def show_annotation_status(model_name, top_cutoff, concept_cutoff):
    annotation_dict = get_annotation_status(model_name, top_cutoff, concept_cutoff)
    # same category not annotated:
    print('completed:\n')
    for prop in sorted(list(annotation_dict['complete'])):
        # cats open:
        print(prop)
    print()
    print('Incomplete:\n')
    for prop in sorted(annotation_dict['incomplete']):
        if prop not in annotation_dict['complete']:
            print(prop)
    return annotation_dict
            
            
def get_evidence_dict(model_name, prop, top_cutoff, concept_cutoff):
    
    annotation_name = f'annotation-tfidf-top_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    f_annotation = f'{path_dir_annotation}/annotation-updated-done.csv'
    
    ev_dict = dict()
    
    with open(f_annotation) as infile:
        data = list(csv.DictReader(infile))
    for d in data:
        et = d['evidence_type']
        ev = d['context']
        ev_dict[ev] = et
    return ev_dict
        
            

def get_evidence_distribution(model_name, prop, top_cutoff, concept_cutoff):
    
    # current file:
    annotation_name = f'annotation-tfidf-top_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    path_dir_annotation = f'../analysis/{model_name}/{annotation_name}/{prop}'
    f_annotation = f'{path_dir_annotation}/annotation-updated-done.csv'
    
    ev_dict = get_evidence_dict(model_name, prop, top_cutoff, concept_cutoff)
    
    ev_cnts = Counter()
    
    for e, et in ev_dict.items():
        ev_cnts[et] += 1
        if et != 'u':
            ev_cnts['all'] += 1
        if et in ['p', 'l', 'n']:
            ev_cnts['prop_specific'] += 1
    
    total_contexts = len(ev_dict)
    
    ev_counts_norm = dict()
    for ev, cnt in ev_cnts.items():
        ev_counts_norm[ev]  = cnt/total_contexts
    return ev_counts_norm

In [45]:
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
ann_dict = show_annotation_status(model_name, top_cutoff, concept_cutoff)

completed:

black
female
fly
lay_eggs
used_in_cooking

Incomplete:

blue
cold
dangerous
green
hot
juicy
made_of_wood
red
roll
round
square
sweet
swim
warm
wheels
wings
yellow


In [46]:
model_name = 'giga_full_updated'
properties = ann_dict['complete']
top_cutoff = 3
concept_cutoff = 3

cols = ['property', 'u', 'all', 'prop_specific', 'p', 'n', 'l', 'i', 'r', 'b']
table = []
for prop in properties:
    d = dict()
    d['property'] =  prop
    d.update(get_evidence_distribution(model_name, prop, top_cutoff, concept_cutoff))
    for c in cols:
        if c not in d:
            d[c] = np.nan
    table.append(d)
   
cols = ['property', 'u', 'all', 'prop_specific', 'p', 'n', 'l', 'i', 'r', 'b']
df = pd.DataFrame(table)[cols]
df[cols].sort_values('all', ascending = False).round(3)

,property,u,all,prop_specific,p,n,l,i,r,b
2,used_in_cooking,0.358,0.642,0.010,0.007,0.003,NaN,0.301,0.330,NaN
3,lay_eggs,0.635,0.365,0.027,0.027,NaN,NaN,0.162,0.095,NaN
1,female,0.711,0.289,0.044,NaN,NaN,0.044,0.156,0.022,0.067
4,fly,0.947,0.053,0.008,0.006,0.001,0.001,0.008,0.033,0.003
0,black,0.956,0.044,0.002,0.001,NaN,0.001,0.036,0.007,NaN


## Testing hypotheses

In [15]:
import json
import csv
import os
from collections import Counter, defaultdict
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [54]:
def load_prop_data(prop):
    
    path = f'../data/aggregated_semantic_info/{prop}.json'
    with open(path) as infile:
        concept_dict = json.load(infile)
    return concept_dict


def load_concept_evidence(concept, prop, model_name, categories):
    
    categories.add('all')
    contexts = set()
    dir_path = f'../results/{model_name}/tfidf-raw-10000/each_target_vs_corpus_per_category'
    
    for cat in categories:
        f_path = f'{dir_path}/{prop}/{cat}/pos/{concept}.csv'
        if os.path.isfile(f_path):
            with open(f_path) as infile:
                data = list(csv.DictReader(infile))
            for d in data:
                context = d['']
                diff = float(d['diff'])
                if diff > 0:
                    contexts.add(context)
    return contexts  

def get_categories(prop, model_name):
    analysis_type = 'tfidf-raw-10000/each_target_vs_corpus_per_category'
    path_dir = f'../results/{model_name}/{analysis_type}'
    path_dir = f'{path_dir}/{prop}'
    categories = set()
    for d in os.listdir(path_dir):
        categories.add(d)
    return categories


def get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff):
    table = dict()
    aggregation_name = f'aggregated-tfidf-raw-10000-categories'
    categories = get_categories(prop, model_name)
    
    path_dir_agg = f'../analysis/{model_name}/{aggregation_name}/{prop}'
    evidence_dict = get_evidence_dict(model_name, prop, top_cutoff, concept_cutoff)
    
    et_context_dict = defaultdict(set)
    for c, et in evidence_dict.items():
        et_context_dict[et].add(c)
    
    # get top performance per evidence type for each category
    for cat in categories:
        path = path = f'{path_dir_agg}/{cat}.csv'
        # load file containing all concepts and simply load first one
        with open(path) as infile:
            data = list(csv.DictReader(infile))
        # sort by performance:
        perf_data = defaultdict(list)
        for d in data:
            f1 = d['f1']
            perf_data[f1].append(d)
        perf_ranked = sorted(list(perf_data.keys()), reverse = True)
        for et, contexts in et_context_dict.items():
            for f1 in perf_ranked:
                data = perf_data[f1]
                d_perf = dict()
                for k, v in d.items():
                    if k != 'context':
                        d_perf[k] = round(float(v), 2)
                contexts_ev = set()
                for d in data:
                    context = d['context']
                    if context in contexts:
                        contexts_ev.add(context)
                if contexts_ev:
                    d_perf['n_c'] = len(contexts_ev)
                    d_perf['contexts'] = ' '.join(contexts_ev)
                    table[(cat, et)] = d_perf
                    break
                
    return table

In [56]:
prop = 'lay_eggs'
model_name = 'giga_full_updated'
top_cutoff = 3
concept_cutoff = 3
table = get_top_ev_categories(prop, model_name, top_cutoff, concept_cutoff)

df = pd.DataFrame(table)
df.T

p     r    f1 n_pos total_pos n_neg total_neg n_c  \
measure       i  0.11  0.33  0.17     1         1     2         2   8   
              r     1     1     1     1         1     0         2   2   
              u     1     1     1     1         1     0         2  15   
              p     1     1     1     1         1     0         2   2   
mammal        i  0.98   0.7  0.81     1         1    15        49   1   
              r     1     1     1     1         1     0        49   1   
              u     1     1     1     1         1     0        49   2   
              p     1     1     1     1         1     0        49   2   
bird          i  0.16  0.05  0.08     1        16     4         4   2   
              r   0.9   0.8  0.82    12        16     0         4   1   
              u   0.9   0.8  0.82    12        16     0         4   6   
              p  0.79  0.65  0.68    10        16     1         4   1   
relation      i  0.55  0.38   0.4     1         2     4         6   6   
              r     1     1     1     2         2     0         6   3   
              u     1     1     1     2         2     0         6   5   
              p     1     1     1     2         2     0         6   1   
all           i  0.86  0.84  0.83    20        33     1        57   1   
              r  0.85   0.8  0.78    15        33     0        57   1   
              u  0.82   0.8  0.78    17        33     2        57   1   
              p  0.85   0.8  0.78    15        33     0        57   1   
food          i  0.15  0.06  0.08     1        13     4         4   1   
              r  0.92  0.88  0.89    11        13     0         4   1   
              u     1     1     1    13        13     0         4   1   
              p  0.84  0.82  0.83    11        13     1         4   1   
communication i  0.88   0.5  0.54     1         1     3         5  10   
              r     1     1     1     1         1     0         5   5   
              u     1     1     1     1         1     0         5  19   
              p     1     1     1     1         1     0         5   1   
animal        i  0.86  0.84  0.83    20        33     1        57   1   
              r  0.85   0.8  0.78    15        33     0        57   1   
              u  0.82   0.8  0.78    17        33     2        57   1   
              p  0.85   0.8  0.78    15        33     0        57   1   
no-cat        i     1     1     1     6         6     0         6   2   
              r     1     1     1     6         6     0         6   2   
              u  0.93  0.92  0.92     5         6     0         6   2   
              p     1     1     1     6         6     0         6   1   
fish          i  0.09  0.06  0.07     1        11     6         6   2   
              r  0.95  0.94  0.94    10        11     0         6   3   
              u  0.95  0.94  0.94    10        11     0         6   5   
              p  0.73  0.65  0.65     6        11     1         6   1   
object        i  0.86  0.84  0.83    20        33     1        57   1   
              r  0.85   0.8  0.78    15        33     0        57   1   
              u  0.82   0.8  0.78    17        33     2        57   1   
              p  0.85   0.8  0.78    15        33     0        57   1   

                                                          contexts  
measure       i      turtles bird goose duck trout salmon fish cod  
              r                                       fishing nest  
              u  wild habitat begins seafood banned preserve sm...  
              p                                           egg eggs  
mammal        i                                               duck  
              r                                         freshwater  
              u                                        begins rare  
              p                                           egg eggs  
bird          i                                          duck bird  
              r                             

## Relation analysis

In [87]:


def load_evidence(model_name, prop, evidence_labels, top_cutoff, concept_cutoff):
    
    context_concept_dict = defaultdict(set)
    
    # get evidence words
    dir_analysis = f'../analysis/{model_name}'
    dir_ann = f'{dir_analysis}/annotation-tfidf-top_{top_cutoff}_{concept_cutoff}-raw-10000-categories'
    path_ann = f'{dir_ann}/{prop}/annotation-updated-done.csv'
    
    with open(path_ann) as infile:
        data = list(csv.DictReader(infile))
    ev_words = [d['context'] for d in data if d['evidence_type'] in evidence_labels]
    
    # load concepts associated with the evidence words
    dir_res = f'../results/{model_name}'
    categories = get_categories(prop, model_name)
    for cat in categories:
        dir_res_p =  f'{dir_res}/tfidf-raw-10000/each_target_vs_corpus_per_category/{prop}/{cat}/pos'
        for c_f in os.listdir(dir_res_p):
            path = f'{dir_res_p}/{c_f}'
            concept = c_f.split('.')[0]
            with open(path) as infile:
                data = list(csv.DictReader(infile))
            contexts = [d[''] for d in data if float(d['diff']) > 0 and d[''] in ev_words]
            for c in contexts:
                context_concept_dict[c].add(concept)
    return context_concept_dict



In [88]:
prop = 'black'
top_cutoff = 3
concept_cutoff = 3
rel_neg = {'implied', 'afforded_unusual', 'variability_open', 'typical_of_concept'}
evidence_labels = {'p', 'n', 'l'}
c_pos, c_neg  = get_hyp_pos_neg(prop, rel_neg)
evidence_dict = load_evidence(model_name, prop, evidence_labels, top_cutoff, concept_cutoff)


In [91]:
hyp_test_concept_dict = defaultdict(dict)
n_concepts_total = len(c_pos) + len(c_neg)
for ev_w, concepts in evidence_dict.items():
    concepts_pos = len(concepts.intersection(c_))
    concepts_pos_norm = concepts_pos/len(c_pos)
    concepts_neg = len(concepts.intersection(c_neg))
    concepts_neg_norm = concepts_neg/len(c_neg)
    print(ev_w)
    print('pos', concepts_pos, concepts_pos_norm)
    print('neg', concepts_neg, concepts_neg_norm)

black
pos 38 0.5277777777777778
neg 12 0.6666666666666666
burned
pos 25 0.3472222222222222
neg 13 0.7222222222222222


In [61]:
properties = ['black','lay_eggs']

model_name = 'giga_full_updated'


# get all pairs in which the only relation > 0.5 is implied
# get all pos pairs that do not have implied > 0.5
# get all pos pairs that do not have implied and have some relations leading to mentions
# compare prop mentions

relations_m = {'typical_of_property', 'variability_limited', 'affording_activity', 'afforded_usual'}
relations_nm = {'typical_of_concept', 'implied_category', 'variability_open', 'afforded_usual'}

target = 'implied_category'
target_pairs = set()
comparison_pairs = set()
comparison_pairs_clean = set()


#properties = get_properties()
for prop in properties:
    concept_dict = load_prop_data(prop)
    for c, d in concept_dict.items():
        ml_label = d['ml_label']
        if ml_label in ['all', 'all-some', 'some', 'few-some']:
            relations = d['relations']
            relations_r = [r for r, v in relations.items() if v > 0.5]
            if len(relations_r) == 1 and target in relations_r:
                target_pairs.add((prop, c))
            elif target not in relations_r:
                comparison_pairs.add((prop, c))
            elif any([r in relations_m for r in relations_r]):
                comparison_pairs_clean.add((prop, c))
            
print(len(target_pairs), len(comparison_pairs), len(comparison_pairs_clean))          

3 78 82


In [62]:
target_pairs

{('lay_eggs', 'crane'), ('lay_eggs', 'flounder'), ('lay_eggs', 'neritidae')}

In [63]:
comparison_pairs_clean

{('black', 'bat'),
 ('black', 'civet'),
 ('black', 'coal'),
 ('black', 'crow'),
 ('black', 'currawong'),
 ('black', 'magpie'),
 ('black', 'night'),
 ('black', 'pit'),
 ('black', 'raisin'),
 ('black', 'starling'),
 ('black', 'tar'),
 ('black', 'tyre'),
 ('lay_eggs', 'acaridae'),
 ('lay_eggs', 'albatross'),
 ('lay_eggs', 'apodidae'),
 ('lay_eggs', 'avocet'),
 ('lay_eggs', 'bass'),
 ('lay_eggs', 'calidris'),
 ('lay_eggs', 'char'),
 ('lay_eggs', 'charadrius'),
 ('lay_eggs', 'chicken'),
 ('lay_eggs', 'clupeidae'),
 ('lay_eggs', 'cotingidae'),
 ('lay_eggs', 'creeper'),
 ('lay_eggs', 'crocodile'),
 ('lay_eggs', 'crow'),
 ('lay_eggs', 'curlew'),
 ('lay_eggs', 'currawong'),
 ('lay_eggs', 'duck'),
 ('lay_eggs', 'egret'),
 ('lay_eggs', 'emberizidae'),
 ('lay_eggs', 'fringillidae'),
 ('lay_eggs', 'grackle'),
 ('lay_eggs', 'gyrfalcon'),
 ('lay_eggs', 'halfbeak'),
 ('lay_eggs', 'haliaeetus'),
 ('lay_eggs', 'halibut'),
 ('lay_eggs', 'hen'),
 ('lay_eggs', 'icteridae'),
 ('lay_eggs', 'laridae'),
 ('lay

In [186]:
# check how often direct mentions of property words are mentioned 
# in the context of the target pairs vs the rest